In [1]:
import pandas as pd
df = pd.read_parquet('../sources/metadata.parquet', engine='pyarrow')
df_large = pd.read_parquet('../sources/metadata-large.parquet', engine='pyarrow')


# read excel_artist_names
import my_utils
artist_info = pd.read_csv("../sources/artist_info.csv", sep="\t")

for col in artist_info.columns:
    if "Unnamed" in col:
        artist_info.drop(col, axis=1, inplace=True)

In [2]:
print(artist_info.head())
print(artist_info.tail())

      lastname     firstname               cp_name  \
0        Aalto         Alvar           Alvar Aalto   
1       Aarons          Slim           Slim Aarons   
2        Abbey  Edwin Austin    Edwin Austin Abbey   
3  Abercrombie      Gertrude  Gertrude Abercrombie   
4    Abramović        Marina      Marina Abramović   

                          tags pseudonyms  
0  architecture, high contrast        NaN  
1  vibrant, photography, scene        NaN  
2               scene, realism        NaN  
3            surrealism, dark,        NaN  
4           scene, photography        NaN  
        lastname firstname           cp_name                        tags  \
3472   Underwood    George  George Underwood                         NaN   
3473     Vivanco     Kelly     Kelly Vivanco                         NaN   
3474     Volegov  Vladimir  Vladimir Volegov  scene, portrait, landscape   
3475       Wrona  Angelina    Angelina Wrona                         NaN   
3476  Zouravliov     Vania  Van

In [3]:
import numpy as np

print(artist_info.head())

artist_dict = {}
# keys are the copy_paste names of the artists
# values are lists of aliases of the artists

for index, row in artist_info.iterrows():
    artist_name = row.cp_name.strip(" ")
    artist_dict[artist_name] = [artist_name]
    
    if row.pseudonyms is not np.nan:
        for pseudonym in row.pseudonyms.split(","):
            pseudonym = pseudonym.strip()
            #print(f'{row.cp_name} {pseudonym}')
            artist_dict[artist_name].append(pseudonym)
        print(f'an artist with multiple names {artist_dict[artist_name]}')
        

      lastname     firstname               cp_name  \
0        Aalto         Alvar           Alvar Aalto   
1       Aarons          Slim           Slim Aarons   
2        Abbey  Edwin Austin    Edwin Austin Abbey   
3  Abercrombie      Gertrude  Gertrude Abercrombie   
4    Abramović        Marina      Marina Abramović   

                          tags pseudonyms  
0  architecture, high contrast        NaN  
1  vibrant, photography, scene        NaN  
2               scene, realism        NaN  
3            surrealism, dark,        NaN  
4           scene, photography        NaN  
an artist with multiple names ['Sabbas Apterus', 'Peter Polach']
an artist with multiple names ['Ron Arad', 'Giorgione']
an artist with multiple names ['Alexander Archipenko', 'Olexandr Archipenko']
an artist with multiple names ['Artgerm', 'Stanley Lau']
an artist with multiple names ['Ralph Bakshi', 'Giorgio Barbarelli da Castelfranco']
an artist with multiple names ['Artur Bordalo', 'Bordalo II']
an artis

In [4]:
print(f'number of artists: {len(artist_dict.keys())}')

number of artists: 3477


## Timeframe analysis

In [5]:
print(df_large.timestamp.min())
print(df_large.timestamp.max())

2022-08-06 21:59:00+00:00
2022-08-20 13:33:00+00:00


## User analysis

In [6]:
print(f'amount of unique users {df_large.user_name.unique().shape}')

amount of unique users (10381,)


In [7]:
print(df_large.shape)
print(df_large.user_name.value_counts())

(14000000, 13)
89bebf4f200a853ac9a7b4dfec1edb160ade299fd60db12d0752bbfd882c6f45    224010
deleted-account                                                      93543
481f5d1579a33041518382518a5e108cfc1fea95f880d081023f533eb1afd82a     49577
5477db661bbe6ff9fd509daf812fe50af4b6216f2b296c3cbf91c6b9e528fdce     38137
ac2524ee4169c7f644f783de87f9bbfb96fa68b6e116dfb1f8b65274edaee97a     37524
                                                                     ...  
46b8b7b984d1d18b88cfcbaeac0728bb5aec9ef5c21e28831e8677d0621ed62e         1
19aba8d820816e84f8866140660d476c013858c2b6407b98bc5db41218095ee4         1
49788eeed56abc8f376073ce17f8097fdbdd3f70274ae2496c34f9543686a865         1
d6473a7a475e429100b5178a0298ebdfdf8d8d27bfc6eceed9f5a8d148c4a4b4         1
5e2d286338eb8cad8abb71c28c4d10cde5e4777a5ab8f9e39021c940b5de613f         1
Name: user_name, Length: 10381, dtype: int64


In [8]:
pd.set_option('display.max_colwidth', None)

print(df_large[df_large.user_name == "89bebf4f200a853ac9a7b4dfec1edb160ade299fd60db12d0752bbfd882c6f45"].head(10))

                                    image_name  \
531  5c91c55e-2997-4c8f-a352-d7a55c9101ad.webp   
532  23756e3c-a208-4b22-9da0-389a20fc8fd1.webp   
533  c74875dd-4fca-4d3c-b72b-cd1857386c1c.webp   
534  dba6e669-c61f-4858-bdea-fa1c1632b772.webp   
535  a78de037-1e01-449a-80d9-09fac1957160.webp   
536  6027269e-c39a-4f7f-a06a-4a2f6b57d7e3.webp   
537  a61eb078-41ce-4358-ba3e-2402c61a32b4.webp   
538  827309b2-38cd-4fdd-a627-042e50fc5f8a.webp   
539  03015b95-b837-43f7-a5e5-6cf1814016b6.webp   
540  0924f662-e63f-4d6e-8757-ac2e061cf273.webp   

                                                                                                                                                                                                                                        prompt  \
531  mount epic expedition bloodswarmer screenshot world of warcraft mounts worldboss bioluminecant light concept sketch art and by feng zhu alena aenami artworks in 4 k beeple, by thomas kinkade heartstone 

In [9]:
df_large.shape

(14000000, 13)

In [10]:
len(df_large["prompt"].unique())

1819808

In [11]:
# save to csv for human readability
df.to_csv("../sources/metadata.csv", escapechar = "\\")
df_large.to_csv("../sources/metadata-large.csv", escapechar = "\\")

# Exact matching

Exact matching by name and known pseudonyms.

## Extract artists + styles from unique Prompts

In [12]:
style_names= my_utils.read_lines_as_list("../sources/styles.txt")
print(f'number of styles: {len(style_names)}')
# print(f'style names: {style_names}')

number of styles: 192


In [ ]:
# Unique prompts
cols = ["prompt", "image_name","user_name"]

extract_artists_styles = df_large[cols].copy()
extract_artists_styles = extract_artists_styles.drop_duplicates(subset="prompt")

extract_artists_styles['artists'] = extract_artists_styles['prompt'].map(
        lambda p: my_utils.extract_artists_exact(p, artist_dict))
extract_artists_styles["num_artists"] = extract_artists_styles['artists'].map(
        lambda p: len(p))

print("artists extracted")
print(extract_artists_styles.head())

extract_artists_styles['styles'] = extract_artists_styles['prompt'].map(
        lambda p: my_utils.extract_styles_exact(p, style_names))
extract_artists_styles["num_styles"] = extract_artists_styles['styles'].map(
        lambda p: len(p))


print(extract_artists_styles.head())
extract_artists_styles.to_parquet("../results/artists_exact_match_large.parquet")
extract_artists_styles.to_csv("../results/artists_exact_match_large.csv", escapechar="\\")
print(f'Artists extracted')

In [ ]:
extract_artists_styles.sort_values(["num_styles"], ascending=False, inplace=True)
print(f'There are some prompts with a lot of mentioned styles:\n{extract_artists_styles.num_styles.describe()}')
print(extract_artists_styles.head())

## Count Artist Mentions

In [ ]:
import pandas as pd
import my_utils
import time

exact_matches = pd.read_parquet('../results/artists_exact_match_large.parquet', engine='pyarrow')
print(f'exact_matches.dtypes {exact_matches.dtypes}')

artist_names = artist_dict.keys()

cols = ["artist", "mentions", "style_mentions"]

artist_mentions =  pd.DataFrame(columns=cols)

starttime = time.time()

for index, name in enumerate(artist_names):
    
    new_row = dict()
    new_row['artist'] = name
    mdf = my_utils.exact_match_dataframe(exact_matches, name)
    new_row['mentions'] = mdf.shape[0]
    new_row['style_mentions'] = mdf.num_styles.sum() # style_mentions sum of all mentioned styles in prompts that contain the artist
    
    new_row = pd.Series(new_row)
    
    artist_mentions = pd.concat([artist_mentions, new_row.to_frame().T], ignore_index=True)
    
    duration = time.time() - starttime
    
    if index % 1000 == 0:
        c = index + 1
        print(f'analysing {c} artists took {duration / 60} minutes',flush=True)
        print(f'time remaining estimate {(duration/c)*(len(artist_names)-c)/60} minutes',flush=True)

artist_mentions.sort_values("mentions", axis=0, ascending=False, inplace=True)
    
artist_mentions.to_csv("../results/artist_mentions.csv", escapechar = "\\")
artist_mentions.to_parquet("../results/artist_mentions.parquet")
print("finished")

# Count Style Mentions

In [ ]:
import pandas as pd
import my_utils
import time

style_names= my_utils.read_lines_as_list("../sources/styles.txt")

exact_matches = pd.read_parquet('../results/artists_exact_match_large.parquet', engine='pyarrow')
print(exact_matches[exact_matches.num_styles > 0].head())


cols = ["style", "mentions"]

style_mentions =  pd.DataFrame(columns=cols)

starttime = time.time()

for index, name in enumerate(style_names):
    
    new_row = dict()
    new_row['style'] = name
    mdf = my_utils.exact_match_dataframe_style(exact_matches, name)
    #print(mdf)
    new_row['mentions'] = mdf.shape[0]
    #print(new_row['mentions'])
    
    new_row = pd.Series(new_row)
    #print(new_row)
    
    style_mentions = pd.concat([style_mentions, new_row.to_frame().T], ignore_index=True)
    
    duration = time.time() - starttime

style_mentions.sort_values("mentions", axis=0, ascending=False, inplace=True)
style_mentions.reset_index(inplace=True)
style_mentions.drop("index", axis=1, inplace=True)

assert style_mentions[style_mentions["mentions"]>0].shape[0]>0, f'No style mentions have been found, check the data.'

print(style_mentions.head())

style_mentions.to_csv("../results/style_mentions.csv", escapechar = "\\")
style_mentions.to_parquet("../results/style_mentions.parquet")

print(f'{style_mentions[style_mentions.mentions > 0].shape[0]} of {len(style_names)} styles have been found')

print("finished")

In [ ]:

print(style_mentions.columns)


assert style_mentions[style_mentions["mentions"]>0].shape[0]>0, f'No style mentions have been found, check the data.'

print(style_mentions.head())